# AWS SETUP

In [1]:
import os
import boto3
import re
import copy
import time
from time import gmtime, strftime
#from sagemaker import get_execution_role

role = "arn:aws:iam::599281969166:role/service-role/AmazonSageMaker-ExecutionRole-20190716T181300"
#region = boto3.Session().region_name

bucket_name='finaldebatebucket' # Replace with your s3 bucket name
prefix = 'sagemaker/FinalProject' # Used as part of the path in the bucket where you store data
bucket_path = 'https://s3-us-east-2.amazonaws.com/finaldebatebucket' # The URL to access the bucket
sub_path = "cache-data"

s3 = boto3.resource('s3')
try:
    if  region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

# Start Notebook

## Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import requests

import boto3
import tpclean.tpclean as tp

from os import listdir
#from __future__ import print_function
from datetime import datetime

## ETL

The goal here is to:
1. go through the Folder of soundfiles to, 
2. get the files trancriped from audio to text
3. transform the text to meaningful metadata aswell as the content
4. load everything into a database

### Extract

`bucket_path`must be : 'https://s3-us-east-2.amazonaws.com/finaldebatebucket'

In [3]:
from Scripts.etl_functions import find_audios , transcribe_wav , get_pause

In [4]:
#load Bucket Content
audio_files = find_audios(bucket_name)
audio_files

['AM_101_affirmativeaction_pro.wav',
 'DJ_1_ban-video-games_pro.wav',
 'EH_1_ban-video-games_pro.wav']

In [5]:
#just for testing here uncomment as soon as not needed
filename = audio_files[1]

In [6]:
#load API 

job_uri = f"{bucket_path}/{sub_path}/{filename}"

In [7]:
trans_json_uri = transcribe_wav(job_uri)[1]

File already transcribed
Output succesfull


### Transform

Now that the desired files are extracted, they can be transformed to get meaningful metadata and the content of the speech

In [8]:
#load json from URL
r = requests.get(trans_json_uri)

#store json
explore = r.json()

#store full text
fulltext = explore["results"]["transcripts"][0]["transcript"]

In [9]:
#create Dataframe
df = pd.DataFrame(explore["results"]["items"])
df.head()

,alternatives,end_time,start_time,type
0,"[{'confidence': '1.0000', 'content': 'We'}]",1.45,1.14,pronunciation
1,"[{'confidence': '1.0000', 'content': 'should'}]",1.69,1.45,pronunciation
2,"[{'confidence': '0.9717', 'content': 'ban'}]",2.16,1.69,pronunciation
3,"[{'confidence': '1.0000', 'content': 'the'}]",2.32,2.17,pronunciation
4,"[{'confidence': '1.0000', 'content': 'sale'}]",2.77,2.32,pronunciation


the nested structure of the dataframe needs to be unnested

In [10]:
#unnest the data using tpclean
df = tp.unnest_df_list(df,["alternatives"])
df = tp.unnest_df_dict(df,["alternatives_1"])
df.rename({"alternatives_1_confidence":"confidence", 
           "alternatives_1_content": "content"}, 
          axis = "columns", inplace = True)

Check the reusults:

In [11]:
df.head()

,end_time,start_time,type,confidence,content
0,1.45,1.14,pronunciation,1.0000,We
1,1.69,1.45,pronunciation,1.0000,should
2,2.16,1.69,pronunciation,0.9717,ban
3,2.32,2.17,pronunciation,1.0000,the
4,2.77,2.32,pronunciation,1.0000,sale


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 5 columns):
end_time      526 non-null object
start_time    526 non-null object
type          577 non-null object
confidence    577 non-null object
content       577 non-null object
dtypes: object(5)
memory usage: 22.6+ KB


In order to properly calculate features columns with numbers must be interpreted as those. As seen above this is currently not the case.

In [13]:
#convert columns containing numbers into float datatype
for col in df.columns:
    try:
        df[col] = df[col].astype("float")
    except:
        continue

Compute Features

In [14]:
#engineer length of word and pauses between words
df["length"] = df.end_time-df.start_time
get_pause(df,"start_time","end_time");

#append filename
df["origin"] = filename

#append default speaker for now
df["speaker"] = "speaker_default"

#append word 
df = df.reset_index().rename({"index":"pos_in_conv"},axis = "columns");

Check the results

In [15]:
df.head()

,pos_in_conv,end_time,start_time,type,confidence,content,length,pause_after,origin,speaker
0,0,1.45,1.14,pronunciation,1.0000,We,0.31,0.00,DJ_1_ban-video-games_pro.wav,speaker_default
1,1,1.69,1.45,pronunciation,1.0000,should,0.24,0.00,DJ_1_ban-video-games_pro.wav,speaker_default
2,2,2.16,1.69,pronunciation,0.9717,ban,0.47,0.01,DJ_1_ban-video-games_pro.wav,speaker_default
3,3,2.32,2.17,pronunciation,1.0000,the,0.15,0.00,DJ_1_ban-video-games_pro.wav,speaker_default
4,4,2.77,2.32,pronunciation,1.0000,sale,0.45,0.00,DJ_1_ban-video-games_pro.wav,speaker_default


### Load

In the final step of the ETL Process, the data is uploaded to a mySQL Database running as an RDS instance on AWS

In [16]:
#setting up connection parameters
from Private.private import user
from Private.private import password

conn_kwargs = {"host":'debaterdb.c7oenlqovcjd.us-east-2.rds.amazonaws.com', 
               "user":user, 
               "password":password}
db = "debater"

In [17]:
#connecting via sqlalchemy because pandas needs an engine to store data in an mysql DB
from sqlalchemy import create_engine
engine = create_engine(f'mysql+pymysql://{user}:{password}@{conn_kwargs["host"]}:3306/{db}')

#connecting via mysql-connector to examine exsisting data
import mysql.connector
conn = tp.sql_connect(db,db_type="mysql", ** conn_kwargs)

Connecting to mysql
successfully imported module


InterfaceError: 2003: Can't connect to MySQL server on 'debaterdb.c7oenlqovcjd.us-east-2.rds.amazonaws.com:3306' (60 Operation timed out)

In [ ]:
tp.sql("""show tables""",df_return = True)

In [ ]:
#check whether file already in the DB
files_in_content = set(tp.sql("SELECT * FROM content").origin.unique())

if not len(set(df.origin.unique()).intersection(files_in_content)):
    #append data to DB
    df.to_sql("content",engine, if_exists="append")
else:
    print("File transcription already in Database -> will not append!")

In [ ]:
tp.sql("SELECT * FROM content").head()

In [ ]:
columns = ", ".join(tp.sql("DESCRIBE conversations", df_return=True).Field.values[1:])
name_split = filename[:-4].split("_")
values = [filename,
                name_split[-2].lower(), 
                int(name_split[-1].lower() == "pro"), 
                trans_json_uri, 
                fulltext]

In [ ]:
#columns

In [ ]:
#values

In [ ]:
querry = """INSERT INTO conversations({}) VALUES ("{}funf","{}",{},"{}","{}");""".format(columns,*values)

In [ ]:
querry

In [ ]:
tp.sql(querry)

In [ ]:
tp.sql("""SELECT * FROM conversations""")

In [ ]:
conn.close()

In [ ]:
conn = tp.sql_connect(db,db_type="mysql", ** conn_kwargs)